In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import numpy as np
from matplotlib.collections import PathCollection
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

import statsmodels.stats.multicomp as mc

In [2]:
# Estadistica
def estadistica_dataFrame(dataframe, columnaValores, columnaCategorical):
    
    import statsmodels.stats.multicomp as mc
    from scipy import stats
    
    comp1 = mc.MultiComparison(dataframe[columnaValores], dataframe[columnaCategorical])
    tbl, a1, a2 = comp1.allpairtest(stats.ttest_ind, method= "bonf")

    comp1_pvalores = pd.DataFrame(a2)
    a10 = pd.DataFrame(a1[0],columns=['stats','p_value'])
    a12 = pd.DataFrame(a1[2], columns=['pvalue_corr'])

    comp1_pvalores = pd.concat([comp1_pvalores,a10,a12], axis=1)

    comp1_pvalores['pval_corr_log10'] = comp1_pvalores['pvalue_corr'].apply(np.log10)
    comp1_pvalores['pval_corr_log10'] = comp1_pvalores['pval_corr_log10'].abs()
    
    return comp1_pvalores

In [3]:
r = pd.read_csv('motivos_TARGETSCAN_conservEVOL_sinsequencia_3UTR_correlaciones_filtrado80porciento.csv', sep='\t',\
                index_col=0)
joinned = pd.DataFrame(r)

/home/juan/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
#valor de expresion de los miRs
#expr = 0
#lista_conservados_expresion = list(merge[merge['media_EXPR'] >= expr]['miR'])

#parametros del for loop

#multiple_boxplot_SOLOunodelosmotivos = pd.DataFrame()
multiple_boxplot_SOLOunodelosmotivos_lista = []
estadisticaDF =[]
estadistica_Ttest = []

mir_numeroDatos_un_motivo = {}
dif_medias_solounmotivo = {}


for miR in joinned['miroRNA_name'].unique():
    
    #Generar el df para cada miR
    
    df = joinned.copy()
    
    df = df[(df['miroRNA_name'] == miR) & (df['MIMAT'].str.contains('_zscore'))] #me quedo con el miR y los valores de ZSCORE o spearman
    df.dropna(subset=['motivo'], inplace=True)
    
    print('----------------------------------------------')
    print(miR,'\n', df.shape)
    
    #saco posibles duplicados dentro de cada motvio (no hay)
    df.sort_values('apariciones_motivo', ascending=False).drop_duplicates(['gen_name', 'motivo'], inplace=True, keep= 'first') #no deberian haber porque los filtre antes, en el script anterior.
    
    #elimino todos los genes que tienen mas de un motivo de cualquiera de ellos, esta rebuscado hay formas mas sencillas de hacerlo pero ahora no voy a tocar el script.
    #Saco todo gen que de cualquiera de los motivos tenga mas de uno.  Saco por el nombre no por la cantidad. Por eso esta bien el filtro.
    df_mayora1 = df[df['apariciones_motivo'] > 1]
    df = df[~df['gen_name'].isin(list(df_mayora1['gen_name']))]
    
    print(miR,'\n', df.shape)
    
    #Agarro todo los motivos con suppl. Incluso creo que este paso no es necesario.
    
    df_s6m4_LaVenganza = df[df['motivo'].str.contains('SUPPL')] 
    print('Df de los df_s6m4_LaVenganza \n', df_s6m4_LaVenganza.shape)
    #Saco si estan duplicados, es decir si estan en mas de uno de los motivos
    df_s6m4_LaVenganza.drop_duplicates(['gen_name'], inplace=True, keep=False)
    #df_s6m4_LaVenganza = df_s6m4_LaVenganza[df_s6m4_LaVenganza['motivo'].str.contains('SUPPL')] #agregado
    
    print('Df de los df_s6m4_LaVenganza \n', df_s6m4_LaVenganza.shape)
    
    
    #Hago lo mismo para los mers, saco cualquier duplicado dentro de ellos, es importante separar porque los suppl y los mers estan duplicados porque los suppl incluyen obviamente a los mers respectivos
    df_mers = df[df['motivo'].isin(['6mer_site','7merA1_site', '7merm8_site', '8mer_site'])]
    
    print('Df de los mers \n',df_mers.shape)
    
    # Me quedo con los nombres de aquellos genes que estan duplicados para luego quitarlos del DF final
    duplicados_mers = set(df_mers[df_mers.duplicated(subset=['gen_name'], keep=False)]['gen_name'])
    
    print(len(duplicados_mers))
    #df_mers.drop_duplicates(['gen_name'], inplace=True, keep=False) #Script viejo
    
    print('Df de los mers \n',df_mers.shape)
    #Ahora que saque los duplicados de los suppl y tengo la lista de los duplicados en mers. Uno los DFs.
    df = pd.concat([df_s6m4_LaVenganza, df_mers])
    
    print('DataFrameConcatenado\n', df.shape)
    
    #saco los duplicados que habia obtenido de los mers    
    df = df[~df['gen_name'].isin(duplicados_mers)]
    #Ahora en el DF los suppl estan arriba, como tiene seguro un duplicado de mers porque lo incluyen lo que hago es sacarduplicados pero quedarme con el de arriba.
    df.drop_duplicates(['gen_name'], inplace=True, keep='first')
    
    print('DataFrameConcatenado sin dupl\n', df.shape)
    #multiple boxplot
    
    #df.loc[:,'microRNA'] = str(miR)
    
    
    #df.rename(columns={eje_y[0]: "correlacion"},inplace=True)#, inplace=True)
    #df.rename(columns={eje_y[1]: "zscore"}, inplace=True)
    
    multiple_boxplot_SOLOunodelosmotivos_lista.append(df)
    #multiple_boxplot_SOLOunodelosmotivos = pd.concat([multiple_boxplot_SOLOunodelosmotivos, df], sort=False)
    
    #statistics
    comp1 = mc.MultiComparison(df['CorrOrZscore'], df['motivo'])
    #tbl, a1, a2 = comp1.allpairtest(stats.ttest_ind, method= "bonf")
    tbl = comp1.tukeyhsd()
    stast_DF = pd.DataFrame(data=tbl._results_table.data[1:], columns=tbl._results_table.data[0])
    stast_DF['microRNA'] = miR
    
    estadisticaDF.append(stast_DF)

    
    #statistics 2
    
    stast2_DF = estadistica_dataFrame(df, 'CorrOrZscore', 'motivo')
    stast2_DF['microRNA'] = miR    
    
    
    estadistica_Ttest.append(stast2_DF)

    

    
multiple_boxplot_SOLOunodelosmotivos = pd.concat(multiple_boxplot_SOLOunodelosmotivos_lista)
dataFrame_estadistica = pd.concat(estadisticaDF)
Student_test_DF = pd.concat(estadistica_Ttest)



----------------------------------------------
hsa-let-7a-5p 
 (6507, 11)
hsa-let-7a-5p 
 (5382, 11)
Df de los df_s6m4_LaVenganza 
 (286, 11)
Df de los df_s6m4_LaVenganza 
 (280, 11)
Df de los mers 
 (5096, 11)
864
Df de los mers 
 (5096, 11)
DataFrameConcatenado
 (5376, 11)
DataFrameConcatenado sin dupl
 (3253, 11)


<ipython-input-4-31de93231e4c>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s6m4_LaVenganza.drop_duplicates(['gen_name'], inplace=True, keep=False)


----------------------------------------------
hsa-let-7b-5p 
 (6507, 11)
hsa-let-7b-5p 
 (5382, 11)
Df de los df_s6m4_LaVenganza 
 (286, 11)
Df de los df_s6m4_LaVenganza 
 (280, 11)
Df de los mers 
 (5096, 11)
864
Df de los mers 
 (5096, 11)
DataFrameConcatenado
 (5376, 11)
DataFrameConcatenado sin dupl
 (3253, 11)


<ipython-input-4-31de93231e4c>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s6m4_LaVenganza.drop_duplicates(['gen_name'], inplace=True, keep=False)


----------------------------------------------
hsa-let-7c-5p 
 (6507, 11)
hsa-let-7c-5p 
 (5382, 11)
Df de los df_s6m4_LaVenganza 
 (286, 11)
Df de los df_s6m4_LaVenganza 
 (280, 11)
Df de los mers 
 (5096, 11)
864
Df de los mers 
 (5096, 11)
DataFrameConcatenado
 (5376, 11)
DataFrameConcatenado sin dupl
 (3253, 11)


...

<ipython-input-4-31de93231e4c>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s6m4_LaVenganza.drop_duplicates(['gen_name'], inplace=True, keep=False)


----------------------------------------------
hsa-miR-187-3p 
 (2061, 11)
hsa-miR-187-3p 
 (1933, 11)
Df de los df_s6m4_LaVenganza 
 (83, 11)
Df de los df_s6m4_LaVenganza 
 (83, 11)
Df de los mers 
 (1850, 11)
127
Df de los mers 
 (1850, 11)
DataFrameConcatenado
 (1933, 11)
DataFrameConcatenado sin dupl
 (1586, 11)


<ipython-input-4-31de93231e4c>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s6m4_LaVenganza.drop_duplicates(['gen_name'], inplace=True, keep=False)


----------------------------------------------
hsa-miR-451a 
 (1525, 11)
hsa-miR-451a 
 (1444, 11)
Df de los df_s6m4_LaVenganza 
 (59, 11)
Df de los df_s6m4_LaVenganza 
 (59, 11)
Df de los mers 
 (1385, 11)
69
Df de los mers 
 (1385, 11)
DataFrameConcatenado
 (1444, 11)
DataFrameConcatenado sin dupl
 (1245, 11)


<ipython-input-4-31de93231e4c>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s6m4_LaVenganza.drop_duplicates(['gen_name'], inplace=True, keep=False)
/home/juan/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/juan/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
multiple_boxplot_SOLOunodelosmotivos.to_csv('Df_unSoloMotivo_TARGETSCAN_zscore_3UTR_correlaciones_filtrado80porciento.csv', sep='\t')
dataFrame_estadistica.to_csv('Df_unSoloMotivo_TARGETSCAN_zscore_tukethsdTEST_3UTR_correlaciones_filtrado80porciento.csv', sep='\t')
Student_test_DF.to_csv('Df_unSoloMotivo_TARGETSCAN_zscore_studentTEST_3UTR_correlaciones_filtrado80porciento.csv', sep='\t')

In [6]:
#dataFrame_estadistica[(dataFrame_estadistica['group1'] == '6mer_site') & (dataFrame_estadistica['group2'] == '8mer_site') & (dataFrame_estadistica['reject'] == True)]

In [7]:
#multiple_boxplot_SOLOunodelosmotivos.to_csv('df_multiple_boxplot_SOLOunodelosmotivos_s8_s7m4_s6m4_s7_s6_3UTR_CambioNDatos_12_2020.csv', sep='\t')